# ANLI Baseline with LLM

You have to implement in this notebook a baseline for ANLI classification using an LLM.
This baseline must be implemented using DSPy.



In [ ]:
# Configure the DSPy environment with the language model - for grok the parameters must be:
# env variable should be in os.environ['XAI_API_KEY']
# "xai/grok-3-mini"
# "gemini/gemini-2.5-flash"
import os
import dspy

In [21]:
with open("grok_key.ini") as f:
        for line in f:
            if "XAI_API_KEY" in line and not line.strip().startswith("#"):
                key_value = line.strip().split("=")
                if len(key_value) == 2:
                    os.environ["XAI_API_KEY"] = key_value[1].split()[0]

with open("gemini_key.ini") as f:
        for line in f:
            if "GEMINI_API_KEY" in line and not line.strip().startswith("#"):
                key_value = line.strip().split("=")
                if len(key_value) == 2:
                    os.environ["GEMINI_API_KEY"] = key_value[1].split()[0]

In [48]:


lm = dspy.LM('xai/grok-3-mini', api_key=os.environ['XAI_API_KEY'])
# for ollama 
# lm = dspy.LM('ollama_chat/devstral', api_base='http://localhost:11434', api_key='')
dspy.configure(lm=lm)

In [69]:
from typing import Literal

#defining the module
class anli_classification_signature(dspy.Signature):

    """Lable the relationship between given premise and hypothesis."""
    
    premise: str = dspy.InputField()
    hypothesis: str = dspy.InputField()
    label: Literal['entailment', 'contradiction', 'neutral'] = dspy.OutputField()
    reason: str = dspy.OutputField()

classify = dspy.ChainOfThought(anli_classification_signature)


## Load ANLI dataset

In [32]:
from datasets import load_dataset

dataset = load_dataset("facebook/anli")
dataset = dataset.filter(lambda x: x['reason'] != None and x['reason'] != "")

In [33]:
dataset

DatasetDict({
    train_r1: Dataset({
        features: ['uid', 'premise', 'hypothesis', 'label', 'reason'],
        num_rows: 2923
    })
    dev_r1: Dataset({
        features: ['uid', 'premise', 'hypothesis', 'label', 'reason'],
        num_rows: 1000
    })
    test_r1: Dataset({
        features: ['uid', 'premise', 'hypothesis', 'label', 'reason'],
        num_rows: 1000
    })
    train_r2: Dataset({
        features: ['uid', 'premise', 'hypothesis', 'label', 'reason'],
        num_rows: 4861
    })
    dev_r2: Dataset({
        features: ['uid', 'premise', 'hypothesis', 'label', 'reason'],
        num_rows: 1000
    })
    test_r2: Dataset({
        features: ['uid', 'premise', 'hypothesis', 'label', 'reason'],
        num_rows: 1000
    })
    train_r3: Dataset({
        features: ['uid', 'premise', 'hypothesis', 'label', 'reason'],
        num_rows: 13375
    })
    dev_r3: Dataset({
        features: ['uid', 'premise', 'hypothesis', 'label', 'reason'],
        num_rows: 1200


## Evaluate Metrics

Let's use the huggingface `evaluate` package to compute the performance of the baseline.


In [34]:
from evaluate import load

accuracy = load("accuracy")
precision = load("precision")
recall = load("recall")
f1 = load("f1")


In [35]:
import evaluate
clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])

In [36]:
clf_metrics.compute(predictions=[0, 1, 0], references=[0, 1, 1])

{'accuracy': 0.6666666666666666,
 'f1': 0.6666666666666666,
 'precision': 1.0,
 'recall': 0.5}

## Your Turn

Compute the classification metrics on the baseline LLM model on each test section of the ANLI dataset for samples that have a non-empty 'reason' field.

You also must show a comparison between the DeBERTa baseline model and this LLM baseline model. The comparison metric should compute the agreement between the two models:
* On how many samples they are both correct [Correct]
* On how many samples Model1 is correct and Model2 is incorrect [Correct1]
* On how many samples Model1 is incorrect and Model2 is correct [Correct2]
* On how many samples both are incorrect [Incorrect]

In [ ]:
from dspy.teleprompt import BootstrapFewShotWithRandomSearch
import random

label_map = {
    "entailment": 0,
    "neutral": 1,
    "contradiction": 2
}

#defining metric for evaluation
def binary_metrics(example, prediction, trace=None):
    print(example.label)
   # print(prediction["label"])
    return example.label == label_map[prediction["label"]]

sample_dev_r3 = random.sample(list(dataset['dev_r3']), 50)

example_trainset = []
for example in sample_dev_r3:
    example_trainset.append(dspy.Example(premise=example["premise"], hypothesis=example["hypothesis"], label=example["label"], reason=example["reason"]).with_inputs("premise", "hypothesis"))




0
0
1
0
1
2
0
0
1
1
0
2
0
2
1
2
0
1
2
1
2
1
0
1
1
0
1
0
0
1
1
2
2
1
1
1
0
0
1
0
2
2
2
1
2
1
0
2
0
1


In [112]:

#optimizing based on dev-r3

dspy.configure_cache(
    enable_disk_cache=True,
    enable_memory_cache=True,
)

config = dict(max_bootstrapped_demos=4, max_labeled_demos=4, num_candidate_programs=10, num_threads=4)

teleprompter = BootstrapFewShotWithRandomSearch(metric=binary_metrics, **config)
optimized_program = teleprompter.compile(classify, trainset=example_trainset)



Going to sample between 1 and 4 traces per predictor.
Will attempt to bootstrap 10 candidate sets.
Average Metric: 0.00 / 1 (0.0%):   0%|          | 0/50 [00:00<?, ?it/s]0
1
Average Metric: 10.00 / 17 (58.8%):  32%|███▏      | 16/50 [00:00<00:00, 84.33it/s]12

Average Metric: 22.00 / 34 (64.7%):  68%|██████▊   | 34/50 [00:17<00:12,  1.25it/s]1
1
Average Metric: 33.00 / 50 (66.0%): 100%|██████████| 50/50 [00:41<00:00,  1.20it/s]

2025/08/06 11:57:41 INFO dspy.evaluate.evaluate: Average Metric: 33 / 50 (66.0%)



New best score: 66.0 for seed -3
Scores so far: [66.0]
Best score so far: 66.0
Average Metric: 10.00 / 15 (66.7%):  30%|███       | 15/50 [00:28<00:53,  1.53s/it]2
0
Average Metric: 36.00 / 50 (72.0%): 100%|██████████| 50/50 [01:15<00:00,  1.51s/it]


2025/08/06 11:58:57 INFO dspy.evaluate.evaluate: Average Metric: 36 / 50 (72.0%)


New best score: 72.0 for seed -2
Scores so far: [66.0, 72.0]
Best score so far: 72.0


...

1


 10%|█         | 5/50 [00:27<04:03,  5.42s/it]


2
Bootstrapped 4 full traces after 5 examples for up to 1 rounds, amounting to 5 attempts.
Average Metric: 13.00 / 17 (76.5%):  34%|███▍      | 17/50 [00:31<01:11,  2.15s/it]21

Average Metric: 37.00 / 50 (74.0%): 100%|██████████| 50/50 [01:15<00:00,  1.52s/it]

2025/08/06 12:16:35 INFO dspy.evaluate.evaluate: Average Metric: 37 / 50 (74.0%)



Scores so far: [66.0, 72.0, 74.0, 62.0, 72.0, 74.0, 72.0, 64.0, 72.0, 74.0, 76.0, 64.0, 74.0]
Best score so far: 76.0
13 candidate programs found.


In [122]:
#next on the agenda: 
#create a list of evaluations - pred/gold labels - on test_r3 ( same as 1.2 baseline)

import json

with open("pred_test_r3.json", "r") as f:
    pred_test_r3 = json.load(f)


In [113]:
print(optimized_program(premise=pred_test_r3[1]['premise'], hypothesis=pred_test_r3[1]['hypothesis']).label)

entailment


In [ ]:
evaluation_list = []

for item in pred_test_r3:
    premise_ = item['premise']
    hypothesis_ = item['hypothesis']
    prediction_llm = optimized_program(premise = premise_, hypothesis = hypothesis_)
    print(prediction_llm.reason)
    evaluation_list.append({
        'premise': premise_,
        'hypothesis': hypothesis_,
        'pred_llm_label': prediction_llm.label,
        'pred_baseline_model_label': item['pred_label'],
        'gold_label': item['gold_label'],
        'reason_llm': prediction_llm.reason,
        'reason_baseline_model': item['reason'],
        'CoT_reasoning': prediction_llm.reasoning
    })


The premise only references Sunday in the context of reviewing posts and does not mention or imply any religious activities, making the hypothesis unrelated and unverified.
The premise confirms that all passengers and crew survived, which means no one, including any potential children, was killed in the accident.
The premise indicates marketing efforts and product history but does not confirm that Japanese people like Kit Kat, leaving the hypothesis unverified.
The premise explicitly states different times for the memorial program (9 a.m.) and the officers' action (10 a.m.), directly implying that the shows of support are not simultaneous.
The premise describes Pietro Grassano's appointment as new, implying he was not previously the country head, which directly contradicts the hypothesis.
The premise's emphasis on the unprecedented and pure presentation of operatic music entails that Fresca Opera is unique.
The premise specifies that the stock fall was due to speculation about curbing 

In [118]:
#Compare the results with the baseline and provide agreement metrics between the two models.
from sklearn.metrics import confusion_matrix, accuracy_score, cohen_kappa_score, classification_report, ConfusionMatrixDisplay

gold_labels = [label_map[e["gold_label"]] for e in evaluation_list]
llm_model_labels = [label_map[e["pred_llm_label"]] for e in evaluation_list]
base_line_model_labels = [label_map[e["pred_baseline_model_label"]] for e in evaluation_list]

acc_llm = accuracy_score(gold_labels, llm_model_labels)
acc_baseline = accuracy_score(gold_labels, base_line_model_labels)

kappa = cohen_kappa_score(llm_model_labels, base_line_model_labels)

print(f"LLM Model Accuracy:     {acc_llm:.3f}")
print(f"Baseline Model Accuracy:{acc_baseline:.3f}")
print(f"Cohen's Kappa Score:    {kappa:.3f}")

print("LLM Model Report:")
print(classification_report(gold_labels, llm_model_labels, target_names=["entailment", "neutral", "contradiction"]))

print("Baseline Model Report:")
print(classification_report(gold_labels, base_line_model_labels, target_names=["entailment", "neutral", "contradiction"]))

# LLM vs Gold
#disp_llm = ConfusionMatrixDisplay.from_predictions(gold_labels, llm_model_labels, display_labels=["entailment", "neutral", "contradiction"], cmap="Blues")
#disp_llm.ax_.set_title("LLM Confusion Matrix")

# Baseline vs Gold
#disp_baseline = ConfusionMatrixDisplay.from_predictions(gold_labels, base_line_model_labels, display_labels=["entailment", "neutral", "contradiction"], cmap="Purples")
#disp_baseline.ax_.set_title("Baseline Confusion Matrix")

# LLM vs Baseline (Agreement Matrix)
#disp_agreement = ConfusionMatrixDisplay.from_predictions(llm_model_labels, base_line_model_labels, display_labels=["entailment", "neutral", "contradiction"], cmap="Greens")
#disp_agreement.ax_.set_title("Model Agreement Confusion Matrix")

LLM Model Accuracy:     0.723
Baseline Model Accuracy:0.495
Cohen's Kappa Score:    0.286
LLM Model Report:
               precision    recall  f1-score   support

   entailment       0.89      0.70      0.78       402
      neutral       0.59      0.80      0.68       402
contradiction       0.79      0.67      0.72       396

     accuracy                           0.72      1200
    macro avg       0.75      0.72      0.73      1200
 weighted avg       0.75      0.72      0.73      1200

Baseline Model Report:
               precision    recall  f1-score   support

   entailment       0.56      0.57      0.56       402
      neutral       0.43      0.50      0.46       402
contradiction       0.51      0.42      0.46       396

     accuracy                           0.49      1200
    macro avg       0.50      0.49      0.49      1200
 weighted avg       0.50      0.49      0.49      1200



In [120]:
with open("evaluation_list.json", "w") as f:
    json.dump(evaluation_list, f, indent=2)